# <p align ="center"><span style='color:navy'> **Formula 1 Stats** </span> </p>
---


### <span style="color:dimgrey; font-family:WildWest;font-size:20px "> I've created this notebook as an alternative to regular statistics </span>
***

- I wanted to move away from the AWS metrics we get on race day and look at answering ~awkward~ insightful queries on anything I can think of. As an alternative this notebook has been built using  [![Azure](https://badgen.net/badge/icon/azure?icon=azure&label)](https://azure.microsoft.com/) cloud services. 

- I've used the free Azure SQL Database and linked it using Azure Data Studio to upload all the CSV files and then query using this notebook. All the queries are done using `T-SQL`
- I'll document them here, if people want to suggest more they are welcome.





<p align="center">
  <img  src="file://D:\git\f1_stats\images\abed-ismail-yasMarina-unsplash_small.jpg" width = 1000>
</p>

Photo by [Abed Ismail](https://unsplash.com/@abedismail?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText) on [Unsplash](https://unsplash.com/s/photos/formula-1?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText)



##### The following Tables are part of the database


|No  |          Tables           |
|:--:|:--------------------:|
|1.  |  Circuits                 | 
|2.  |  Constructor Results      | 
|3.  |  Constructor Standings    | 
|4.  |  Constructors             | 
|5.  |  Driver Standings         |   
|6.  |  Drivers                  |   
|7.  |  Lap Times                |  
|8.  |  Pit Stops                |
|9.  |  Qualifying               |
|10. |  Races                    |
|11. |  Results                  |
|12. |  Seasons                  | 
|13. |  Status                   |



This dataset has been created from the [Formula 1 World Championship (1950-2021)](https://www.kaggle.com/rohanrao/formula-1-world-championship-1950-2020) on Kaggle, the data was derived from the [Ergast API](http://ergast.com/mrd/) 


##  <p align="center">**1.** <span style='color:steelblue'>Who is the most sucessful driver in a season? </span></p>
---

- With an increasing amount of races in the F1 calendar the easiest way to compare efforts across the seasons is to use percentages
- Although who's to say Alberto Ascari couldn't have kept up that 75% win ratio across a 20 race season?

In [13]:

SELECT TOP 10 ra.year as Season ,count(*) as Wins , CONCAT(forename, ' ', UPPER(surname)) as Driver, sum(count(*)) OVER(partition by ra.year) as Races, CONCAT(ROUND(CAST(count(*) as float)/sum(count(*)) OVER(partition by ra.year)*100,2),'%') as [Win Percentage]
FROM [form].[results] re 
JOIN [form].[races] ra 
ON re.raceId = ra.raceId
JOIN [form].[drivers] d
ON re.driverId = d.driverId
WHERE position = 1
GROUP BY ra.year, CONCAT(forename, ' ', UPPER(surname))
ORDER BY CAST(count(*) as float)/sum(count(*)) OVER(partition by ra.year)*100 DESC



(10 rows affected)

Total execution time: 00:00:00.033

Season,Wins,Driver,Races,Win Percentage
1952,6,Alberto ASCARI,8,75%
2004,13,Michael SCHUMACHER,18,72.22%
1963,7,Jim CLARK,10,70%
2013,13,Sebastian VETTEL,19,68.42%
1954,6,Juan FANGIO,9,66.67%
2002,11,Michael SCHUMACHER,17,64.71%
2020,11,Lewis HAMILTON,17,64.71%
1965,6,Jim CLARK,10,60%
2011,11,Sebastian VETTEL,19,57.89%
2014,11,Lewis HAMILTON,19,57.89%


## Just for completeness....

### Here's the most wins in a single season

In [15]:
SELECT TOP 10 ra.year as Season ,count(*) as Wins , CONCAT(forename, ' ', UPPER(surname)) as Driver, sum(count(*)) OVER(partition by ra.year) as Races, CONCAT(ROUND(CAST(count(*) as float)/sum(count(*)) OVER(partition by ra.year)*100,2),'%') as [Win Percentage]
FROM [form].[results] re 
JOIN [form].[races] ra 
ON re.raceId = ra.raceId
JOIN [form].[drivers] d
ON re.driverId = d.driverId
WHERE position = 1
GROUP BY ra.year, CONCAT(forename, ' ', UPPER(surname))
ORDER BY count(*) DESC

(10 rows affected)

Total execution time: 00:00:00.032

Season,Wins,Driver,Races,Win Percentage
2004,13,Michael SCHUMACHER,18,72.22%
2013,13,Sebastian VETTEL,19,68.42%
2002,11,Michael SCHUMACHER,17,64.71%
2011,11,Sebastian VETTEL,19,57.89%
2014,11,Lewis HAMILTON,19,57.89%
2018,11,Lewis HAMILTON,21,52.38%
2019,11,Lewis HAMILTON,21,52.38%
2020,11,Lewis HAMILTON,17,64.71%
2015,10,Lewis HAMILTON,19,52.63%
2016,10,Lewis HAMILTON,21,47.62%


# <p align="center">**2.** <span style='color:steelblue'>What about the most successful Constructor?</span> </p>

  

- ### probably an easy one for fans with McLaren winning 15 of the 16 in 1988, but did you expect Alfa Romeo to be in the Top 10?

In [1]:
with cte as (SELECT  ra.year as Season ,count(*) as Wins , c.name as Constructor
FROM [form].[results] re 
JOIN [form].[races] ra 
ON re.raceId = ra.raceId
JOIN [form].[constructors] c
ON re.constructorId = c.constructorId
WHERE position = 1
GROUP BY ra.year, c.name
)

SELECT TOP 10 Season, Wins, Constructor, sum(Wins) OVER(partition by Season) as Races, ROUND(CAST(Wins as float)/sum(Wins) OVER(partition by Season)*100,2) as [Win Percentage]
FROM cte
ORDER BY CAST(Wins as float)/sum(Wins) OVER(partition by Season)*100   DESC ; 

(10 rows affected)

Total execution time: 00:00:00.087

Season,Wins,Constructor,Races,Win Percentage
1988,15,McLaren,16,93.75
2016,19,Mercedes,21,90.48
2002,15,Ferrari,17,88.24
1952,7,Ferrari,8,87.5
1950,6,Alfa Romeo,7,85.71
2014,16,Mercedes,19,84.21
2015,16,Mercedes,19,84.21
2004,15,Ferrari,18,83.33
1953,7,Ferrari,9,77.78
2020,13,Mercedes,17,76.47


In [14]:
with cte as (SELECT  ra.year as Season ,count(*) as Wins , c.name as Constructor
FROM [form].[results] re 
JOIN [form].[races] ra 
ON re.raceId = ra.raceId
JOIN [form].[constructors] c
ON re.constructorId = c.constructorId
WHERE position = 1
GROUP BY ra.year, c.name
)

SELECT TOP 10 CONCAT(Constructor, ': ', Season), ROUND(CAST(Wins as float)/sum(Wins) OVER(partition by Season)*100,2) as [Win Percentage]
FROM cte
ORDER BY CAST(Wins as float)/sum(Wins) OVER(partition by Season)*100   DESC ; 

(10 rows affected)

Total execution time: 00:00:00.037

(No column name),Win Percentage
McLaren: 1988,93.75
Mercedes: 2016,90.48
Ferrari: 2002,88.24
Ferrari: 1952,87.5
Alfa Romeo: 1950,85.71
Mercedes: 2014,84.21
Mercedes: 2015,84.21
Ferrari: 2004,83.33
Ferrari: 1953,77.78
Mercedes: 2020,76.47


## <p align='center'>**3.** <span style='color:steelblue'>Championships thrown away on the final Race? </span></p>
---

- Which drivers left the championship leaders ruing a Silver? :)

In [16]:
with pen as (SELECT ds.driverId, d.driverRef, year, points, ds.raceId, RANK() OVER(partition by year order by points DESC) AS pen_race_ranking
FROM form.driver_standings as DS
JOIN form.drivers AS d ON ds.driverId = d.driverId 
JOIN form.races AS r on r.raceid = ds.raceId
WHERE ds.raceId IN (SELECT MAX(r1.raceId)-1 FROM form.races as r1 GROUP BY year)),
--ORDER BY year DESC
ult as (SELECT ds.driverId, d.driverRef, year, MAX(points) as final_points,  RANK() OVER(partition by year order by max(points) DESC) as final_standings
FROM form.driver_standings as DS
JOIN form.drivers AS d ON ds.driverId = d.driverId 
JOIN form.races AS r on r.raceid = ds.raceId

GROUP BY ds.driverId, d.driverRef, year)
--ORDER by year desc)

SELECT p.year,UPPER(p.driverRef) as Driver,  pen_race_ranking as [Penultimate Championship Position], final_standings as [Final Standings]
FROM pen AS p
JOIN ult as u on p.driverRef= u.driverRef and p.year= u.year
WHERE final_standings = 1 and pen_race_ranking !=1
ORDER BY p.year DESC

Warning: Null value is eliminated by an aggregate or other SET operation.

(10 rows affected)

Total execution time: 00:00:00.325

year,Driver,Penultimate Championship Position,Final Standings
2010,VETTEL,3,1
2007,RAIKKONEN,3,1
1999,HAKKINEN,2,1
1997,VILLENEUVE,2,1
1986,PROST,2,1
1983,PIQUET,2,1
1981,PIQUET,2,1
1976,HUNT,2,1
1964,SURTEES,2,1
1950,FARINA,3,1


### Not going to lie my favourite was the [2007 win](https://en.wikipedia.org/wiki/2007_Formula_One_World_Championship) for Kimi 
<p align="center"> <img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c9/Kimi_Raikkonen.jpg/170px-Kimi_Raikkonen.jpg" > </p>

## However one omission to that was the 1974 Championship

In [12]:
with pen as (SELECT ds.driverId, d.driverRef,CONCAT(forename, ' ', UPPER(surname)) as [Driver], year, points, ds.raceId, RANK() OVER(partition by year order by points DESC) AS pen_race_ranking
FROM form.driver_standings as DS
JOIN form.drivers AS d ON ds.driverId = d.driverId 
JOIN form.races AS r on r.raceid = ds.raceId
WHERE ds.raceId IN (SELECT MAX(r1.raceId)-1 FROM form.races as r1 GROUP BY year)),

ult as (SELECT ds.driverId, d.driverRef, year, MAX(points) as final_points,  RANK() OVER(partition by year order by max(points) DESC) as final_standings
FROM form.driver_standings as DS
JOIN form.drivers AS d ON ds.driverId = d.driverId 
JOIN form.races AS r on r.raceid = ds.raceId

GROUP BY ds.driverId, d.driverRef, year)


SELECT Driver, p.year, pen_race_ranking, final_standings 
FROM pen AS p
JOIN ult as u on p.driverRef= u.driverRef and p.year= u.year
WHERE final_standings >1  and pen_race_ranking = 1 and p.year = 1974


Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.085

Driver,year,pen_race_ranking,final_standings
Clay REGAZZONI,1974,1,2


### <span style='color:#2874A6'>So why was that not included in the first query?</span>

Delving a little deeper we see that it is due to this section of the code:

```
WHERE final_standings = 1 and pen_race_ranking !=1
```

- The error is not so much an error as a _unique_ situation

For the 1974 Season **two drivers** went into the final season of the race tied on points

In [11]:
SELECT TOP 2 CONCAT(forename, ' ', UPPER(surname)) as Driver, year as Season , points, RANK() OVER(partition by year order by points DESC) AS  [Penultimate Race Ranking]
FROM form.driver_standings as DS
JOIN form.drivers AS d ON ds.driverId = d.driverId 
JOIN form.races AS r on r.raceid = ds.raceId
WHERE ds.raceId IN (SELECT MAX(r1.raceId)-1 FROM form.races as r1 GROUP BY year) and year = 1974


(2 rows affected)

Total execution time: 00:00:00.052

Driver,Season,points,Penultimate Race Ranking
Emerson FITTIPALDI,1974,52,1
Clay REGAZZONI,1974,52,1


Fittipaldi emerged victorious

## <p align='center'>**4.** <span style='color:steelblue'> Winning from the back? </span></p>

While they are several drivers that have recovered from dropping down the order. (Jenson Button in Canada 2011), have any done it after qualifying at the back?

In [8]:
with grid_pos as (SELECT ra.raceId, ra.name as Track, year, Max(grid)as last, Max(grid) - 1 as back_row
FROM form.results r
JOIN form.races ra ON r.raceId = ra.raceId

GROUP BY ra.raceId,ra.name, year),
pod AS (SELECT ra.raceid, r.driverId, driverRef, grid, position, year
FROM form.results AS r
JOIN form.races AS ra ON r.raceId = ra.raceId
JOIN form.drivers AS dr ON dr.driverId = r.driverId
WHERE position IN (1,2,3))

SELECT g.Track, g.year, p.grid, UPPER(p.driverRef) as [Driver], position, count(*) over(partition by g.Track) as Times, count(*) over(partition by p.driverRef)
FROM grid_pos AS g
JOIN pod AS p ON p.grid = g.last OR p.grid = g.back_row 
WHERE g.raceId = p.raceId
ORDER BY [year] DESC;

(17 rows affected)

Total execution time: 00:00:00.067

Track,year,grid,Driver,position,Times,(No column name)
Brazilian Grand Prix,2019,20,SAINZ,3,1,1
German Grand Prix,2019,20,VETTEL,2,2,2
Belgian Grand Prix,2016,21,HAMILTON,3,1,2
Hungarian Grand Prix,2014,22,HAMILTON,3,1,2
Abu Dhabi Grand Prix,2012,24,VETTEL,3,1,2
Australian Grand Prix,2009,20,TRULLI,3,1,1
Bahrain Grand Prix,2006,22,RAIKKONEN,3,1,1
German Grand Prix,2005,20,MONTOYA,2,2,1
Canadian Grand Prix,2005,20,BARRICHELLO,3,1,1
United States Grand Prix West,1980,24,EMERSON_FITTIPALDI,3,1,1


Evidently not

  

Which leads to the question, how far down the order can you qualify and expect to win?

In [6]:
WITH driver as (SELECT re.raceId, grid, CONCAT(forename, ' ', UPPER(surname)) as [Driver], position, name, [year]
FROM form.results AS re
JOIN form.drivers AS dr ON re.driverId = dr.driverId
JOIN form.races AS ra ON ra.raceId = re.raceId
WHERE position = 1),
grids as (SELECT max(grid) as bkRow, re.raceid, [year]
FROM form.results AS re 
JOIN form.races AS ra ON re.raceid = ra.raceId
GROUP BY re.raceId, [year])

SELECT TOP 10 WITH TIES grid, Driver, position, g.[year], bkRow- grid as [Grid Places Ahead of Last]
FROM driver AS d
JOIN grids AS g on d.raceId = g.raceId and g.year = d.year
ORDER BY grid DESC;

(14 rows affected)

Total execution time: 00:00:00.071

grid,Driver,position,year,Grid Places Ahead of Last
22,John WATSON,1,1983,4
19,Bill VUKOVICH,1,1954,14
18,Rubens BARRICHELLO,1,2000,4
17,Kimi RAIKKONEN,1,2005,3
17,John WATSON,1,1982,9
16,Jackie STEWART,1,1973,9
16,Michael SCHUMACHER,1,1995,8
15,Fernando ALONSO,1,2008,5
14,Jenson BUTTON,1,2006,8
14,Johnny HERBERT,1,1999,8


## <p align='center'>*5.* <span style='color:steelblue'> Percentage Likelihood of Winning By Qualifying Position </span></p>

- It's _all_ about the Saturday in F1

In [3]:
SELECT grid as [Grid Position],
	ROUND(CAST(count(*) AS FLOAT)/ (SELECT count(*) from form.results WHERE position = 1)*100,2) AS [Percentage Chance]
FROM form.results
WHERE position = 1
GROUP BY grid
ORDER by grid ASC;

(20 rows affected)

Total execution time: 00:00:00.051

Grid Position,Percentage Chance
1,41.93
2,23.97
3,12.32
4,6.11
5,4.58
6,3.63
7,2.1
8,1.53
9,0.38
10,0.96


## <p align='center'><span style='color:steelblue'> *6.* The Effect of the Undercut </span></p>



- One of the most talked about strategies in the modern era the _undercut_ has proved effective at 72 races since 2011 to 2020 (not including the current season)
- Prior to this refuelling may have negated the effect
- The result run is based purely on the initial pit stop

In [21]:
with [1st] as(SELECT re.raceId, [name], driverRef as [number1], lap, [year], [stop] 
FROM form.results AS re
JOIN form.drivers AS dr on dr.driverId = re.driverId
JOIN form.pit_stops AS p on p.driverId = re.driverId and p.raceId = re.raceId
JOIN form.races AS ra ON ra.raceId = re.raceId
WHERE position = 1 and stop=1),
[2nd] as (SELECT re.raceId, driverRef as [number2], lap, [year], [stop] 
FROM form.results AS re
JOIN form.drivers AS dr on dr.driverId = re.driverId
JOIN form.pit_stops AS p on p.driverId = re.driverId and p.raceId = re.raceId
JOIN form.races AS ra ON ra.raceId = re.raceId
WHERE position = 2 and stop =1)

SELECT  [1st].[name] as [Circuit], UPPER(number1) as [P1], [1st].[lap] AS [1:Pitted On], UPPER(number2) as [P2],  [2nd].[lap] AS [2:Pitted On], [1st].[year], count(*) OVER(partition by number1)
FROM [1st]
JOIN [2nd] ON [1st].raceId = [2nd].raceId and [1st].[stop] = [2nd].[stop]
WHERE [1st].lap < [2nd].lap AND [1st].year != 2021
ORDER BY [1st].[year] DESC

(74 rows affected)

Total execution time: 00:00:00.071

Circuit,P1,1:Pitted On,P2,2:Pitted On,year,(No column name)
Italian Grand Prix,GASLY,19,SAINZ,22,2020,1
Portuguese Grand Prix,HAMILTON,40,BOTTAS,41,2020,30
Turkish Grand Prix,HAMILTON,8,PEREZ,10,2020,30
Styrian Grand Prix,HAMILTON,27,BOTTAS,34,2020,30
Hungarian Grand Prix,HAMILTON,3,MAX_VERSTAPPEN,4,2020,30
Sakhir Grand Prix,PEREZ,1,OCON,41,2020,1
Belgian Grand Prix,LECLERC,21,HAMILTON,22,2019,2
Italian Grand Prix,LECLERC,20,BOTTAS,27,2019,2
Brazilian Grand Prix,MAX_VERSTAPPEN,21,GASLY,22,2019,2
Singapore Grand Prix,VETTEL,19,LECLERC,20,2019,15


## <p align='center'><span style='color:steelblue'> *7.* It's All About Consistency </span></p>

- It's not necessarily the guy with the most wins that takes home the championship.
- The following is list of all the times the driver with the most wins didn't have the most points
- The query extracts where most wins did not equal the Drivers Championship, often this has led to drivers being tied on wins, but not on points.

In [7]:
with cte(driverRef, [year], Wins) as(SELECT driverRef, [year],
	SUM (CASE WHEN position = 1 THEN 1 ELSE 0 END)  as Wins
FROM form.results AS re
JOIN form.drivers AS dr ON dr.driverId = re.driverId
JOIN form.races as ra ON ra.raceId = re.raceid
WHERE  re.driverId IN  (SELECT driverId from form.results where position = 1 GROUP BY driverId)
GROUP BY driverRef, [year]), 
cte2 as 
(SELECT * , max(wins) over(partition by [year]) as max_wins FROM cte),

cte3 as(SELECT ds.points, driverRef,[year], 
	RANK() OVER(partition by [year] order by ds.points DESC) as ranking
FROM form.driver_standings as ds 
JOIN form.drivers as dr ON dr.driverId = ds.driverId
JOIN form.races AS ra ON ra.raceId =ds.raceId
join form.results as re ON re.driverId = dr.driverId
GROUP BY ds.points, driverRef, [year])

SELECT c.[year], UPPER(c.driverRef) as [Most Wins], c.Wins, UPPER(c2.driverRef) [f1 Champion], c4.Wins
FROM (SELECT c1.[year], c1.driverRef, c1.Wins FROM cte2 AS c1 WHERE Wins=max_wins) as c
JOIN (SELECT c3.driverRef, c3.[year] from cte3 AS c3 WHERE ranking = 1) as c2
JOIN cte as C4 ON c4.[year] = c2.[year] and c4.driverRef = c2.driverRef
ON c.[year] = c2.[year]
WHERE c.driverRef != c2.driverRef
ORDER BY c.[year] DESC

(29 rows affected)

Total execution time: 00:00:00.771

year,Most Wins,Wins,f1 Champion,Wins
2016,HAMILTON,10,ROSBERG,9
2010,ALONSO,5,VETTEL,5
2008,MASSA,6,HAMILTON,5
2006,MICHAEL_SCHUMACHER,7,ALONSO,7
2005,RAIKKONEN,7,ALONSO,7
1989,SENNA,6,PROST,4
1987,MANSELL,6,PIQUET,3
1986,MANSELL,5,PROST,4
1984,PROST,7,LAUDA,5
1983,PROST,4,PIQUET,3


## <p align='center'><span style='color:steelblue'> *8.* The Lack of Consistency </span></p>

- A list of drivers that, no matter how many times they tried, never made it to the podium

In [12]:
SELECT TOP 10 CONCAT(dr.forename,' ', UPPER(dr.driverRef)), count(driverRef) as [Races] 
FROM form.drivers as dr
JOIN form.results AS r ON r.driverId = dr.driverId
WHERE  dr.driverRef NOT IN (SELECT dr1.driverRef FROM form.drivers as dr1 JOIN form.results as r1 ON r1.driverId = dr1.driverId
WHERE position IN  (1,2,3))
--JOIN form.driver_standings AS ds ON dr.driverId = ds.driverId
--WHERE position NOT IN (1,2,3)
GROUP BY CONCAT(dr.forename,' ', UPPER(dr.driverRef))
ORDER BY count(*) DESC;

(10 rows affected)

Total execution time: 00:00:00.044

(No column name),Races
Nico HULKENBERG,182
Adrian SUTIL,128
Pierluigi MARTINI,123
Philippe ALLIOT,115
Piercarlo GHINZANI,111
Pedro DINIZ,99
Marcus ERICSSON,97
Ukyo KATAYAMA,97
Marc SURER,88
Jonathan PALMER,88


## <p align='center'><span style='color:steelblue'> *9.* Championships by Country; including most chamionships by a driver </span></p>

- Formula 1 for all it's global reach is dominated by European drivers with the spread of championships breaking down like this:


In [2]:
SELECT DISTINCT nationality as Nationality, count(*) OVER(partition by nationality) as [Championships by Country]
FROM form.drivers as dr 
JOIN form.driver_standings as st 
ON dr.driverId = st.driverId
JOIN (SELECT max(raceId) as final, [year]
FROM form.races GROUP BY [year]) as ra
ON st.raceId = ra.final
WHERE position = 1
ORDER BY [Championships by Country] DESC;

(14 rows affected)

Total execution time: 00:00:00.096

Nationality,Championships by Country
British,20
German,12
Brazilian,8
Argentine,5
Australian,4
Austrian,4
Finnish,4
French,4
Italian,3
American,2


In [5]:
with countries as (SELECT DISTINCT nationality, count(*) OVER(partition by nationality) as [Championships by Country]
FROM form.drivers as dr 
JOIN form.driver_standings as st 
ON dr.driverId = st.driverId
JOIN (SELECT max(raceId) as final, [year]
FROM form.races GROUP BY [year]) as ra
ON st.raceId = ra.final
WHERE position = 1),


ranked as (SELECT DISTINCT CONCAT(forename,' ',UPPER(surname)) as Driver, 
	count(*) as [Championships], nationality,  
	RANK() OVER(partition by nationality order by COUNT(*) DESC) as [Rank]
FROM form.drivers as dr 
JOIN form.driver_standings as st 
ON dr.driverId = st.driverId
JOIN (SELECT max(raceId) as final, [year]
FROM form.races GROUP BY [year]) as ra
ON st.raceId = ra.final
WHERE position = 1
GROUP BY CONCAT(forename,' ',UPPER(surname)), nationality)

SELECT c.nationality as Nationality, c.[Championships by Country], Driver as [Most Successful Driver], [Championships]
from countries as c
JOIN ranked as r
ON c.nationality = r.nationality
WHERE [Rank] =1
ORDER BY [Championships by Country] DESC;

(16 rows affected)

Total execution time: 00:00:00.057

Nationality,Championships by Country,Most Successful Driver,Championships
British,20,Lewis HAMILTON,7
German,12,Michael SCHUMACHER,7
Brazilian,8,Ayrton SENNA,3
Brazilian,8,Nelson PIQUET,3
Argentine,5,Juan FANGIO,5
Australian,4,Jack BRABHAM,3
Austrian,4,Niki LAUDA,3
Finnish,4,Mika HAKKINEN,2
French,4,Alain PROST,4
Italian,3,Alberto ASCARI,2
